In [103]:
# Test for api
from openai import OpenAI
from os import environ
import os

# if doesn't work, try to set the api key here
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


# client = OpenAI()
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "write a haiku about ai"}
    ]
)


In [13]:
print(completion.choices[0].message)

ChatCompletionMessage(content='Silent circuits hum,  \nLearning in the shadows deep,  \nWisdom born from code.  ', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


# Personalized Recommendation

In [24]:
import pandas as pd
from tqdm import tqdm

In [75]:
df = pd.read_csv("data/raw_data_sentiment_analysis.csv")

# Level based recommendation

In [82]:
df_example = df.sample(30)
df_example.head(2)

,subreddit,subtopic,user_name,content2,label,score,content,level
2514,cancer,new treatment,fuzzylol7,These are my instructions\n\n[https://imgur.co...,NEG,0.865347,These are my instructions\n\n[https://imgur.co...,strongly negative
1362,science,research study,Deleted,[removed],NEU,0.957009,[removed],neutral


In [84]:
res = []
for index, row in tqdm(df_example.iterrows()):
    subreddit = row["subreddit"]
    subtopic = row["subtopic"]
    content = row["content"]
    level = row["level"]

    if len(content) < 20:
        res.append("Skipped")
        continue
    
    prompt = f"""
# Task
You are a expert consultant with strong background in healthcare representing Turmerik and you are talking to a patient. The patient might indicates an intention to take a clinical trial.
Your task is to generate a personalized response to the patient.
According to our record, the user has a level of interest of {level} in the topic of {subtopic} in the subreddit {subreddit}.

# Context
Here is the comment from the patient that you might consider when generating your response:
<{content}>

# Requirement
You should be friendly, careful and professional when generating your response. Do not force the patient to take the clinical trial rather provide them with the necessary information and let them decide.
Please indicate your role first in the response. 
# Response
"""


    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    res.append(completion.choices[0].message.content)

30it [01:16,  2.56s/it]


In [85]:
res

["Hello! I'm a healthcare consultant with a strong background in oncology and I’m here to provide support and information regarding your situation. \n\nI understand that navigating through cancer treatment options can be overwhelming, especially with the complexities of different diagnostic procedures like PET and PET CT scans. You're absolutely right; they do serve different purposes, and it’s important to clarify these details to ensure you’re making the best-informed decisions for your health.\n\nRegarding clinical trials, I appreciate that you might have concerns given your experiences. It’s completely normal to have reservations about new treatments, especially when you've already been through so much. Clinical trials can offer access to new therapies and potentially contribute to better understanding of your condition, but they’re not the right path for everyone.\n\nIf you’re considering looking into clinical trials, I recommend discussing it with your healthcare provider who can

# Precise recommendation

In [72]:
res = []

for comment in tqdm(df['content'][500:]):
    prompt = """Your task is to classify whether the following text indicates an intention to take a clinical trial. 
    You should output 'yes' if the text indicates an strong intention to take a clinical trial,'maybe' if the text indicate a possible intention to take a clinical trial, 'slightly' if the text indicates a slight possibility to take a clinical trial, and 'no' if the text does not indicate an intention to take a clinical trial.
    Please only output one word: 'yes', 'maybe', 'slightly', or 'no'.
    Here are some examples of output format you need to follow:

    Text: <Can you share your experience in the clinical trials?>
    Answer: yes

    Text: <I suffer from depression all the time however I don't the risk of taking new medication will help.>
    Answer: possibly

    Text: <My friend's taking a clinical trial for diabetes.>
    Answer: slightly

    Text: <Don't trust those doctors.>
    Answer: no

    Now its your turn.
    
    Text: <{comment}>
    Answer:
"""

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    res.append(completion.choices[0].message.content)

100%|██████████| 2212/2212 [14:27<00:00,  2.55it/s]


In [73]:
ps = pd.Series(res)

In [74]:
ps.value_counts()

no                                                                                                                                  2210
Answer: no                                                                                                                             1
I'm sorry, but it looks like the input does not contain any text to classify. Could you please provide a comment for evaluation?       1
Name: count, dtype: int64

In [51]:
print(df.iloc[559]['content2'])

Oh wow, I'm so sorry. Are you comfortable sharing what you experienced?


In [58]:
prompt_personalized = f"""
# Task
You are a expert consultant with strong background in healthcare representing Turmerik and you are talking to a patient. The patient might indicates an intention to take a clinical trial.
Your task is to generate a personalized response to the patient.

# Context
Here is the comment from the patient that you might consider when generating your response:
{df.iloc[559]['content2']}

# Requirement
You should be friendly, careful and professional when generating your response. Do not force the patient to take the clinical trial rather provide them with the necessary information and let them decide.
Please indicate your role first in the response. 
# Response
"""

In [59]:

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": prompt_personalized}
    ]
)
print(completion.choices[0].message.content)

As a healthcare consultant representing Turmerik, I'd be happy to share my experiences with you. It's important for me to convey that participating in clinical trials can come with various factors to consider, both positive and challenging. Each trial has unique protocols, and it's essential to understand how they might affect your health and wellbeing.

If you're considering a clinical trial, I can provide you with detailed information about what participation entails, possible benefits, and any risks involved. This way, you can make an informed decision that feels right for you. If you have any specific questions or concerns, please feel free to ask. I'm here to help!
